In [34]:
import uproot
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Image


masses = [3, 4, 4.5, 5, 7.5, 10, 12.5, 15, 17.5, 20, 22.5]
lifetimes = {1:0, 10:1, 100:2}

def set_bin_content(mass, lifetime, value, arr):
    ybin = np.max(np.digitize(mass, masses)-1, 0)
    xbin = lifetimes[lifetime]
    arr[ybin,xbin] = value

def make_uncertainty_plot(histpath, title, outfile, ax):

    uncertianty_arr = np.array([
    [ 0.0,0.0,0.0,],
    [ 0.0,0.0,0.0,],
    [ 0.0,0.0,0.0,],
    [ 0.0,0.0,0.0,],
    [ 0.0,0.0,0.0,],
    [ 0.0,0.0,0.0,],
    [ 0.0,0.0,0.0,],
    [ 0.0,0.0,0.0,],
    [ 0.0,0.0,0.0,],
    [ 0.0,0.0,0.0,],
    ])

    count_arr = np.array(uncertianty_arr)

    # get bins from ntuples    
    for hist_file in glob.glob(histpath):
        # hacky way to get all MC periods
        # f_mc16e = uproot.open(hist_file)
        # f_mc16d = uproot.open(hist_file.replace('mc16e','mc16d'))
        # f_mc16a = uproot.open(hist_file.replace('mc16e','mc16a'))
        # mean = np.concatenate([
        #     f_mc16e['VSI_LeptonsMod_ntuples_LNC_plus_LNV_mHNL']['d0_extrapolation_1DOWN'].array(),
        #     f_mc16d['VSI_LeptonsMod_ntuples_LNC_plus_LNV_mHNL']['d0_extrapolation_1DOWN'].array(),
        #     f_mc16a['VSI_LeptonsMod_ntuples_LNC_plus_LNV_mHNL']['d0_extrapolation_1DOWN'].array(),
        #     ]).mean()

        f_fullrun2 = uproot.open(hist_file)
        uncertainty_value = -100
        passed_events = -100
        try:
            # take the per-event difference in systematics
            if TREE_SYSTEMATIC:
                nominal = f_fullrun2[f'nominal_VSI_LeptonsMod_ntuples_LNC_plus_LNV_mHNL'][f'HNLm'].array().mean()
                down = f_fullrun2[f'{sys_of_interest}__1up_VSI_LeptonsMod_ntuples_LNC_plus_LNV_mHNL'][f'HNLm'].array().mean()
                up = f_fullrun2[f'{sys_of_interest}__1down_VSI_LeptonsMod_ntuples_LNC_plus_LNV_mHNL'][f'HNLm'].array().mean()
                uncertainty_value = (abs(nominal-up) + abs(nominal-down)) / 2 # average
            else:
                nominal = f_fullrun2['nominal_VSI_LeptonsMod_ntuples_LNC_plus_LNV_mHNL']['SF_nominal'].array()
                down = f_fullrun2['nominal_VSI_LeptonsMod_ntuples_LNC_plus_LNV_mHNL'][f'SF_{sys_of_interest}__1down'].array()
                up = f_fullrun2['nominal_VSI_LeptonsMod_ntuples_LNC_plus_LNV_mHNL'][f'SF_{sys_of_interest}__1up'].array()
                uncertainty_value = ((abs(nominal-up) + abs(nominal-down)) / 2).mean() # average
            # print(nominal[0])
            # print(up[0])
            # return()
            passed_events = len(f_fullrun2['nominal_VSI_LeptonsMod_ntuples_LNC_plus_LNV_mHNL']['SF_nominal'])
        except Exception as e:
            # print('failure', hist_file)
            # print(e)
            pass

        tokens = hist_file.split('/')[5].split('_')
        file_lifetime = int(tokens[3].split('mm')[0])
        file_mass = float(tokens[2][:-1].replace('p','.'))
        set_bin_content(file_mass, file_lifetime, uncertainty_value, uncertianty_arr)
        set_bin_content(file_mass, file_lifetime, passed_events, count_arr)

    # do the plotting
    plot_uncertainties(uncertianty_arr, count_arr, title, outfile, ax)
    return uncertianty_arr, count_arr

def plot_uncertainties(uncertianty_arr, count_arr, title, outfile, ax):
    # plt.figure(figsize=[7,8])
    im = ax.pcolormesh([0,1,2,3], masses, uncertianty_arr, vmin=0, vmax=uncertianty_arr.max()*1.5)
    plt.colorbar(im, ax=ax)
    # ax = plt.gca()

    # mess with x ticks
    ax.set_xticks([0,1,2,3])
    ax.set_xticklabels('')
    ax.set_xticklabels(['1 mm', '10 mm', '100 mm',], minor=True)
    ax.set_xticks([0.5,1.5,2.5,], minor=True)
    ax.tick_params(axis='both', which='minor', length=0)

    # mess with y ticks
    ax.set_yticks(masses)
    ax.set_yticklabels('')
    ax.tick_params(which='minor', length=0)
    ax.set_yticks(masses[:-1] + np.diff(masses)/2, minor=True)
    ax.set_yticklabels(masses[:-1], minor=True)

    ax.set_title(title)
    ax.set_xlabel('lifetime [mm]')
    ax.set_ylabel('mass [GeV]')

    # set text
    for i in range(len(masses)-1):
        for j in range(len(lifetimes)):
            yloc = j+.5
            xloc = masses[i] + (masses[i+1]-masses[i])/2
            if uncertianty_arr[i,j] >= 0:
                text = f'{uncertianty_arr[i,j]:0.2%} ± {uncertianty_arr[i,j]/np.sqrt(count_arr[i,j]):.2%}' # percent uncertainty
                # text = f'{uncertianty_arr[i,j]:0.2%} ({int(count_arr[i,j])})' # nevents used
            else:
                text = 'no sig.'
            ax.text(yloc, xloc , text,
                    ha="center", va="center", color="w", fontweight='bold', transform=ax.transData)

    # plt.savefig('/home/newhouse/public/Analysis/HNL/dhnlanalysisnotebooks/systematics/leptons/plots/png/'+outfile+'.png', dpi=300)
    # plt.savefig('/home/newhouse/public/Analysis/HNL/dhnlanalysisnotebooks/systematics/leptons/plots/pdf/'+outfile+'.pdf', dpi=300)

DO_DISPLAY = False

# TREE_SYSTEMATIC = False
# for sys_of_interest in [
#                         ]:

uncertianty_arr_dict = {}

for sys_of_interest in [# SFs
                        'MUON_EFF_RECO_SYS',
                        'MUON_EFF_RECO_SYS_LOWPT',
                        'MUON_EFF_TrigSystUncertainty',
                        'EL_EFF_Reco_TOTAL_1NPCOR_PLUS_UNCOR',
                        'EL_EFF_ID_TOTAL_1NPCOR_PLUS_UNCOR',
                        'EL_EFF_Trigger_TOTAL_1NPCOR_PLUS_UNCOR',
                        # trees
                        'nominalMUON_ID',
                        'nominalMUON_MS',
                        'nominalMUON_SAGITTA_RESBIAS',
                        'nominalMUON_SAGITTA_RHO',
                        'nominalMUON_SCALE',
                        'nominalEG_RESOLUTION_ALL',
                        'nominalEG_SCALE_AF2',
                        'nominalEG_SCALE_ALL',
                        ]:

    if 'nominal' in sys_of_interest:
        TREE_SYSTEMATIC = True
    else:
        TREE_SYSTEMATIC = False

                        
    fig, axs = plt.subplots(2, 2, figsize=[14,16])

    uncertianty_arr_list = []

    print(sys_of_interest, 'uuu')
    uncertianty_arr, count_arr = make_uncertainty_plot(histpath='/data/hnl/histograms/v9p3_histograms/histograms_fullrun2_*_uuu.root', title=f'{sys_of_interest} μμμ', outfile=f'uuu_{sys_of_interest}', ax=axs[0,0])
    if DO_DISPLAY: display(Image(filename=f'/home/newhouse/public/Analysis/HNL/dhnlanalysisnotebooks/systematics/leptons/plots/png/uuu_{sys_of_interest}.png'));
    uncertianty_arr_list.append(uncertianty_arr)

    print(sys_of_interest, 'uue')
    uncertianty_arr, count_arr = make_uncertainty_plot(histpath='/data/hnl/histograms/v9p3_histograms/histograms_fullrun2_*_uue.root', title=f'{sys_of_interest} μμe', outfile=f'uue_{sys_of_interest}', ax=axs[0,1])
    if DO_DISPLAY: display(Image(filename=f'/home/newhouse/public/Analysis/HNL/dhnlanalysisnotebooks/systematics/leptons/plots/png/uue_{sys_of_interest}.png'));
    uncertianty_arr_list.append(uncertianty_arr)

    print(sys_of_interest, 'eeu')
    uncertianty_arr, count_arr = make_uncertainty_plot(histpath='/data/hnl/histograms/v9p3_histograms/histograms_fullrun2_*_eeu.root', title=f'{sys_of_interest} eeμ', outfile=f'eeu_{sys_of_interest}', ax=axs[1,0])
    if DO_DISPLAY: display(Image(filename=f'/home/newhouse/public/Analysis/HNL/dhnlanalysisnotebooks/systematics/leptons/plots/png/eeu_{sys_of_interest}.png'));
    uncertianty_arr_list.append(uncertianty_arr)

    print(sys_of_interest, 'eee')
    uncertianty_arr, count_arr = make_uncertainty_plot(histpath='/data/hnl/histograms/v9p3_histograms/histograms_fullrun2_*_eee.root', title=f'{sys_of_interest} eee', outfile=f'eee_{sys_of_interest}', ax=axs[1,1])
    if DO_DISPLAY: display(Image(filename=f'/home/newhouse/public/Analysis/HNL/dhnlanalysisnotebooks/systematics/leptons/plots/png/eee_{sys_of_interest}.png'));
    uncertianty_arr_list.append(uncertianty_arr)

    # plt.savefig('/home/newhouse/public/Analysis/HNL/dhnlanalysisnotebooks/systematics/leptons/plots/png/'+sys_of_interest+'.png', dpi=300)
    # plt.savefig('/home/newhouse/public/Analysis/HNL/dhnlanalysisnotebooks/systematics/leptons/plots/pdf/'+sys_of_interest+'.pdf', dpi=300)

    uncertianty_arr_dict[sys_of_interest] = uncertianty_arr_list

# import glob
# import os
# import PyPDF2

# merger = PyPDF2.PdfFileMerger()

# for f in sorted(glob.glob('/home/newhouse/public/Analysis/HNL/dhnlanalysisnotebooks/systematics/leptons/plots/pdf/*'), key=os.path.getmtime):
#     # print(f)
#     merger.append(f)

# merger.write("/home/newhouse/public/Analysis/HNL/dhnlanalysisnotebooks/systematics/leptons/plots/result.pdf")
# merger.close()


MUON_EFF_RECO_SYS_LOWPT uuu


/home/newhouse/.local/lib/python3.7/site-packages/ipykernel_launcher.py:89: UserWarning: FixedFormatter should only be used together with FixedLocator


MUON_EFF_RECO_SYS_LOWPT uue
MUON_EFF_RECO_SYS_LOWPT eeu
MUON_EFF_RECO_SYS_LOWPT eee


In [40]:
# uncertianty_arr_dict_backup = uncertianty_arr_dict
# print(uncertianty_arr_dict_new)
# uncertianty_arr_dict_new.update(/)
uncertianty_arr_dict = uncertianty_arr_dict_new
# import json
# print(json.dumps(uncertianty_arr_dict), file=open('uncertianty_arr_dict.json', 'w'))

# outfile = 'uncertianty_arr_dict.npz'
# np.savez(outfile, **uncertianty_arr_dict)

# npzfile = np.load(outfile)
# print('npzfile.files: {}'.format(npzfile.files))
# print('npzfile["MUON_EFF_RECO_SYS"]: {}'.format(npzfile["MUON_EFF_RECO_SYS"]))


In [24]:
impact_dict = {}
for k, v in uncertianty_arr_dict.items():
    list_of_uncertainties = []
    for a in v:
        # a = vv[0]
        list_of_uncertainties.append(a[a>0].mean())
    impact_dict[k] = np.max(list_of_uncertainties)
print(impact_dict)

{'nominalMUON_ID': 0.003131090647874437}


{'MUON_EFF_RECO_SYS': [array([[-1.00000000e+02, -1.00000000e+02, -1.00000000e+02],
         [-1.00000000e+02, -1.00000000e+02, -1.00000000e+02],
         [-1.00000000e+02, -1.00000000e+02, -1.00000000e+02],
         [-1.00000000e+02, -1.00000000e+02, -1.00000000e+02],
         [ 2.28725752e-03,  2.10071990e-03,  2.14945432e-03],
         [ 2.14318444e-03,  2.11560222e-03,  1.97617407e-03],
         [ 2.22314139e-03,  2.07720974e-03,  1.90895874e-03],
         [ 2.28398309e-03,  2.11169463e-03,  1.93531277e-03],
         [ 2.22584747e-03,  2.09191931e-03,  1.86935452e-03],
         [ 2.55779765e-03,  2.11801182e-03,  2.15002561e-03]]),
  array([[-1.00000000e+02,  1.63905170e-03,  1.12080653e-03],
         [ 4.31474602e-03,  1.56827693e-03,  1.60548579e-03],
         [ 3.79193834e-03,  1.39405785e-03,  1.37646815e-03],
         [ 4.16860628e-03,  1.42085286e-03,  1.52255992e-03],
         [ 1.76067478e-03,  1.50728898e-03,  1.39243276e-03],
         [ 1.49163477e-03,  1.54877528e-03,  1.

In [ ]:
import glob
import os
import PyPDF2

merger = PyPDF2.PdfFileMerger()

for f in sorted(glob.glob('/home/newhouse/public/Analysis/HNL/dhnlanalysisnotebooks/systematics/leptons/plots/pdf/*'), key=os.path.getmtime):
    # print(f)
    merger.append(f)

merger.write("/home/newhouse/public/Analysis/HNL/dhnlanalysisnotebooks/systematics/leptons/plots/systematics_impact.pdf")
merger.close()

In [ ]:
np.sqrt(
    uncertianty_arr_dict['MUON_EFF_RECO_SYS'][0]**2 + 
    uncertianty_arr_dict['MUON_EFF_TrigSystUncertainty'][0]**2
    )

In [81]:
import uproot
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Image

# nominal = uproot.open('/data/hnl/histograms/v9p2_histograms/histograms_fullrun2_10G_10mm_uuu.root')['nominal']['VSI_LeptonsMod_ntuples_LNC_plus_LNV_mHNL'].pandas.df()
nominalMUON_SAGITTA_RESBIAS__1down = uproot.open('/data/hnl/histograms/v9p2_histograms/histograms_fullrun2_10G_10mm_uuu.root')['nominalMUON_MS__1down']['VSI_LeptonsMod_ntuples_LNC_plus_LNV_mHNL']['HNLm'].array()
nominalMUON_SAGITTA_RESBIAS__1up = uproot.open('/data/hnl/histograms/v9p2_histograms/histograms_fullrun2_10G_10mm_uuu.root')['nominalMUON_MS__1up']['VSI_LeptonsMod_ntuples_LNC_plus_LNV_mHNL']['HNLm'].array()
df = pd.DataFrame()
df['nominalMUON_SAGITTA_RESBIAS__1up'] = nominalMUON_SAGITTA_RESBIAS__1up
df['nominalMUON_SAGITTA_RESBIAS__1down'] = nominalMUON_SAGITTA_RESBIAS__1down
df['diff'] = nominalMUON_SAGITTA_RESBIAS__1up - nominalMUON_SAGITTA_RESBIAS__1down
df['diff'].hist(bins=60)
plt.show()
df['diff'].value_counts()

ValueError: Length of values (1510) does not match length of index (1511)

In [40]:
down = uproot.open('/data/hnl/histograms/v9p2_histograms/histograms_fullrun2_10G_1mm_uuu.root')['nominalMUON_MS__1down']['VSI_LeptonsMod_ntuples_LNC_plus_LNV_mHNL'].pandas.df()
up = uproot.open('/data/hnl/histograms/v9p2_histograms/histograms_fullrun2_10G_1mm_uuu.root')['nominalMUON_MS__1up']['VSI_LeptonsMod_ntuples_LNC_plus_LNV_mHNL'].pandas.df()
diff = down - up

In [8]:
[print(x) for x in uproot.open('/data/hnl/histograms/v9p2_histograms/histograms_fullrun2_10G_1mm_uuu.root')['nominal']['VSI_LeptonsMod_ntuples_LNC_plus_LNV_mHNL'].keys()];
b'SF_nominal'
b'SF_MUON_EFF_RECO_SYS__1down'
b'SF_MUON_EFF_RECO_SYS__1up'
b'SF_EL_EFF_Reco_TOTAL_1NPCOR_PLUS_UNCOR__1down'
b'SF_EL_EFF_Reco_TOTAL_1NPCOR_PLUS_UNCOR__1up'
b'SF_EL_EFF_ID_TOTAL_1NPCOR_PLUS_UNCOR__1down'
b'SF_EL_EFF_ID_TOTAL_1NPCOR_PLUS_UNCOR__1up'
b'SF_MUON_EFF_TrigSystUncertainty__1down'
b'SF_MUON_EFF_TrigSystUncertainty__1up'
b'SF_EL_EFF_Trigger_TOTAL_1NPCOR_PLUS_UNCOR__1down'
b'SF_EL_EFF_Trigger_TOTAL_1NPCOR_PLUS_UNCOR__1up'
b'SF_weight_pileup_up'
b'SF_weight_pileup_down'
b'model_weight_one_majorana_hnl_LNCplusLNV_single_flavour_mixing'
b'model_weight_quasi_dirac_pair_LNCplusLNV_ih_mixing'
b'model_weight_quasi_dirac_pair_LNCplusLNV_nh_mixing'
b'model_weight_one_dirac_hnl_LNC_single_flavour_mixing'
b'model_weight_quasi_dirac_pair_LNC_ih_mixing'
b'model_weight_quasi_dirac_pair_LNC_nh_mixing'
b'LNC_xsec_one_majorana_hnl_single_flavour'
b'LNC_xsec_one_dirac_hnl_single_flavour'
b'NH_xsec'
b'IH_xsec'
b'event_is_LNC'
b'event_is_LNV'
b'plep_pt'
b'plep_eta'
b'plep_phi'
b'plep_d0'
b'plep_z0'
b'plep_charge'
b'plep_isTight'
b'plep_is_trigger_matched'
b'mvis'
b'mtrans'
b'HNLm'
b'HNLm_altbinning'
b'alt_HNLm'
b'HNLm_fixWmass'
b'HNLpt'
b'HNLeta'
b'HNLphi'
b'DV_redmass'
b'DV_redmassvis'
b'DV_redmassHNL'
b'mll_dMu_plep_is_OS'
b'mll_dMu_plep_is_SS'
b'mll_dMu_plep'
b'DV_trk_deta'
b'DV_trk_dphi'
b'DV_trk_dpt'
b'DV_trk_dR'
b'DV_cosmic_sep'
b'DV_trk_max_chi2_toSV'
b'DV_trk_min_chi2_toSV'
b'DV_trk_max_d0_wrtSV'
b'DV_trk_min_d0_wrtSV'
b'DV_trk_max_errd0_wrtSV'
b'DV_trk_min_errd0_wrtSV'
b'DV_trk_max_z0_wrtSV'
b'DV_trk_min_z0_wrtSV'
b'DV_trk_max_errz0_wrtSV'
b'DV_trk_min_errz0_wrtSV'
b'DV_mumu'
b'DV_ee'
b'DV_emu'
b'DV_1lep'
b'DV_pass_el_mu_overlap'
b'DV_pass_lep_pt'
b'DV_trk_0_pt'
b'DV_trk_0_eta'
b'DV_trk_0_phi'
b'DV_trk_0_d0'
b'DV_trk_0_z0'
b'DV_trk_0_charge'
b'DV_trk_0_chi2'
b'DV_trk_0_isSelected'
b'DV_trk_0_isAssociated'
b'DV_trk_0_mom_parall'
b'DV_trk_0_mom_perp'
b'DV_trk_0_mom_mag'
b'DV_trk_0_mom_frac_parall'
b'DV_trk_1_pt'
b'DV_trk_1_eta'
b'DV_trk_1_phi'
b'DV_trk_1_d0'
b'DV_trk_1_z0'
b'DV_trk_1_charge'
b'DV_trk_1_chi2'
b'DV_trk_1_isSelected'
b'DV_trk_1_isAssociated'
b'DV_trk_1_mom_parall'
b'DV_trk_1_mom_perp'
b'DV_trk_1_mom_mag'
b'DV_trk_1_mom_frac_parall'
b'DV_mu_0_trk_pt_wrtSV'
b'DV_mu_1_trk_pt_wrtSV'
b'DV_mu_0_std_trk_pt'
b'DV_mu_1_std_trk_pt'
b'DV_mu_0_lepmatched_trk_pt'
b'DV_mu_1_lepmatched_trk_pt'
b'DV_mu_0_lepmatched_trk_eta'
b'DV_mu_1_lepmatched_trk_eta'
b'DV_mu_0_lepmatched_trk_phi'
b'DV_mu_1_lepmatched_trk_phi'
b'DV_mass_lepmatched'
b'DV_mass_diff'
b'DV_mu_0_pt_diff'
b'DV_mu_1_pt_diff'
b'DV_mu_0_pt_diff_lep_matched'
b'DV_mu_1_pt_diff_lep_matched'
b'DV_mu_0_is_trigger_matched'
b'DV_mu_1_is_trigger_matched'
b'DV_mu_0_isMuon'
b'DV_mu_1_isMuon'
b'DV_mu_0_charge'
b'DV_mu_1_charge'
b'DV_mu_0_isElectron'
b'DV_mu_1_isElectron'
b'DV_mu_0_muon_isLoose'
b'DV_mu_1_muon_isLoose'
b'DV_mu_0_muon_isMedium'
b'DV_mu_1_muon_isMedium'
b'DV_mu_0_muon_isTight'
b'DV_mu_1_muon_isTight'
b'DV_mu_trk_pt'
b'DV_mu_trk_eta'
b'DV_mu_trk_phi'
b'DV_trk_pt'
b'DV_trk_eta'
b'DV_trk_phi'
b'DV_trk_d0'
b'DV_trk_z0'
b'DV_trk_absz0'
b'DV_trk_charge'
b'DV_trk_chi2'
b'DV_trk_isLRT'
b'DV_trk_isSelected'
b'DV_trk_isAssociated'
b'DV_trk_nPixelHits'
b'DV_trk_nSCTHits'
b'DV_trk_nSiHits'
b'DV_trk_chi2_toSV'
b'DV_trk_d0_wrtSV'
b'DV_trk_errd0_wrtSV'
b'DV_trk_z0_wrtSV'
b'DV_trk_errz0_wrtSV'
b'DV_num_trks'
b'DV_x'
b'DV_y'
b'DV_z'
b'DV_r'
b'PV_x'
b'PV_y'
b'PV_z'
b'DV_distFromPV'
b'DV_mass'
b'DV_pt'
b'DV_eta'
b'DV_phi'
b'DV_minOpAng'
b'DV_maxOpAng'
b'DV_charge'
b'DV_chi2'
b'DV_max_dR'
b'DV_max_dR_wrtSV'
b'DV_maxd0'
b'DV_mind0'
b'DV_ntrk'
b'DV_ntrk_lrt'
b'DV_ntrk_sel'
b'DV_ntrk_assoc'
b'DV_pass_mat_veto'
b'DV_truth_matched'
b'properLifetime'
b'DV_trk_0_d0_truth'
b'DV_trk_1_d0_truth'
b'DV_trk_v_mu_pt'
b'DV_alpha'
b'DV_2tight'
b'DV_2medium'
b'DV_2loose'
b'DV_1tight'
b'DV_1medium'
b'DV_1loose'
b'DV_tight_loose'
b'DV_tight_medium'
b'DV_medium_loose'
b'DV_tight_veryloose'
b'DV_medium_veryloose'
b'DV_loose_veryloose'
b'DV_tight_veryveryloose'
b'DV_medium_veryveryloose'
b'DV_loose_veryveryloose'
b'DV_2veryveryloose'
b'DV_1veryveryloose'
b'n_trigger_matched_medium'
b'vertexing_1DOWN'
b'd0_extrapolation_1DOWN'


b'SF_nominal'
b'SF_MUON_EFF_RECO_SYS__1down'
b'SF_MUON_EFF_RECO_SYS__1up'
b'SF_EL_EFF_Reco_TOTAL_1NPCOR_PLUS_UNCOR__1down'
b'SF_EL_EFF_Reco_TOTAL_1NPCOR_PLUS_UNCOR__1up'
b'SF_EL_EFF_ID_TOTAL_1NPCOR_PLUS_UNCOR__1down'
b'SF_EL_EFF_ID_TOTAL_1NPCOR_PLUS_UNCOR__1up'
b'SF_MUON_EFF_TrigSystUncertainty__1down'
b'SF_MUON_EFF_TrigSystUncertainty__1up'
b'SF_EL_EFF_Trigger_TOTAL_1NPCOR_PLUS_UNCOR__1down'
b'SF_EL_EFF_Trigger_TOTAL_1NPCOR_PLUS_UNCOR__1up'
b'SF_weight_pileup_up'
b'SF_weight_pileup_down'
b'model_weight_one_majorana_hnl_LNCplusLNV_single_flavour_mixing'
b'model_weight_quasi_dirac_pair_LNCplusLNV_ih_mixing'
b'model_weight_quasi_dirac_pair_LNCplusLNV_nh_mixing'
b'model_weight_one_dirac_hnl_LNC_single_flavour_mixing'
b'model_weight_quasi_dirac_pair_LNC_ih_mixing'
b'model_weight_quasi_dirac_pair_LNC_nh_mixing'
b'LNC_xsec_one_majorana_hnl_single_flavour'
b'LNC_xsec_one_dirac_hnl_single_flavour'
b'NH_xsec'
b'IH_xsec'
b'event_is_LNC'
b'event_is_LNV'
b'plep_pt'
b'plep_eta'
b'plep_phi'
b'plep_d